In [26]:
import pandas as pd

df = pd.read_csv("../Datasets/merged_reviews.csv")
df.head()


,category,label,text,source
0,conrad,truthful,We stayed for a one night getaway with family ...,TripAdvisor
1,hyatt,truthful,Triple A rate with upgrade to view room was le...,TripAdvisor
2,hyatt,truthful,This comes a little late as I'm finally catchi...,TripAdvisor
3,omni,truthful,The Omni Chicago really delivers on all fronts...,TripAdvisor
4,hyatt,truthful,I asked for a high floor away from the elevato...,TripAdvisor


Checking if dataset is balanced

In [46]:
df["binary_label"] = df["label"].map({
    "truthful": "REAL",
    "OR": "REAL",
    "deceptive": "FAKE",
    "CG": "FAKE"
})


In [48]:
print(df["binary_label"].value_counts())

binary_label
REAL    21016
FAKE    21016
Name: count, dtype: int64


Define Output Schema

In [49]:
from pydantic import BaseModel, Field

class ReviewClassification(BaseModel):
    # label: str = Field(description="truthful, deceptive, CG, OR")
    label: str = Field(description="fake or original")
    explanation: str


Create Prompt Template

In [50]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("""
You are an expert in detecting fake, manipulated, or AI-generated reviews.

Classify the following review strictly as one of:
- "fake"
- "original"

Return ONLY valid JSON:
{{
  "label": "fake/original",
  "explanation": "short explanation"
}}

Review:
"{text}"
""")


# prompt = PromptTemplate.from_template("""
# You are an expert at detecting authentic vs fake product reviews.

# Classify the following review into exactly ONE of these four labels:

# 1. "truthful"  = genuine human-written review  
# 2. "deceptive" = intentionally fake or misleading  
# 3. "CG"        = computer-generated or AI-written fake  
# 4. "OR"        = original human-written review (authentic)

# Return ONLY valid JSON with the following structure:
# {{
#   "label": "truthful/deceptive/CG/OR",
#   "explanation": "short justification"
# }}

# Review:
# "{text}"
# """)



Load Model

In [51]:
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(model="gemma:2b")



Build Chain

In [52]:
from langchain_core.output_parsers import JsonOutputParser

parser = JsonOutputParser(pydantic_object=ReviewClassification)

chain = prompt | llm | parser


Test Single Prediction

In [53]:
result = chain.invoke({"text": "Stayed at Fairmont Chicago Millennium Park and it was not a good experience. The room had been cleaned in a haphazardly fashion and was not pleasant to walk into. When I called the service desk for more towels to be brought up into the room, it took an exceedingly long amount of time for them to be brought up, and the woman who delivered the towels was as rude as can be. Will not be staying at Fairmont Chicago Millennium Park again."})
print(result)


{'label': 'fake', 'explanation': 'The review contains subjective and first-person language, which is indicative of a fake review.'}


In [ ]:
predictions = []
explanations = []

for review in df["text"]:
    try:
        out = chain.invoke({"text": review})
        predictions.append(out.label)
        explanations.append(out.explanation)
    except Exception as e:
        predictions.append("error")
        explanations.append(str(e))


Zero-Shot Classification on Entire CSV

In [ ]:
df["predicted_label"] = predictions
df["explanation"] = explanations

Evaluate Performance

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

y_true = df["label"]
y_pred = df["predicted_label"]

print("Accuracy:", accuracy_score(y_true, y_pred))
print(classification_report(y_true, y_pred))

Save Results

In [ ]:
df.to_csv("zero_shot_results_class.csv", index=False)